# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,27.99,78,75,7.72,CC,1699180228
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,0.81,79,75,5.36,RU,1699180369
2,2,adamstown,-25.0660,-130.1015,22.27,76,96,6.69,PN,1699180370
3,3,puerto ayora,-0.7393,-90.3518,24.75,90,78,3.13,EC,1699180372
4,4,ceelbuur,4.6850,46.6176,31.03,54,99,1.91,SO,1699180373


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
import geoviews.tile_sources as gvts


pd.options.plotting.backend = "holoviews"

# Assuming you have a DataFrame 'city_data_df' with columns "Lng", "Lat", "Humidity", "City", and "Country"
df = pd.DataFrame(
    city_data_df,
    columns=[
        "Lng",
        "Lat",
        "Humidity",
        "City",
        "Country",  # Assuming you have a "Country" column
    ],
)

tile_options = {"width": 700, "height": 500, "xaxis": None, "yaxis": None}
world = gvts.OSM.opts(**tile_options)

# Calculate alpha values based on humidity
df["alpha"] = df["Humidity"] / 100  # Adjust scaling as needed

world * df.hvplot.points(
    geo=True, s="Humidity", c="City", alpha="alpha", hover_cols=["City", "Humidity"]
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,alpha)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Define your criteria (example criteria provided)
max_temp_criteria = (city_data_df["Max Temp"] > 21) & (
    city_data_df["Max Temp"] < 27)
cloudiness_criteria = city_data_df["Cloudiness"] < 60
wind_speed_criteria = city_data_df["Wind Speed"] < 10

# Combine all criteria to find cities that meet all conditions
filtered_cities = pd.DataFrame.copy(
    city_data_df[max_temp_criteria & cloudiness_criteria & wind_speed_criteria]
)

# Drop any rows with null values
filtered_cities.dropna(inplace=True)

# Display sample data (first 5 rows)
filtered_cities.head()
print(filtered_cities)

     City_ID                         City      Lat       Lng  Max Temp  \
11        11                     alvaraes  -3.2208  -64.8042     24.30   
36        36                    tazacorte  28.6290  -17.9293     24.54   
46        46                         tezu  27.9167   96.1667     21.21   
58        58                  nova sintra  14.8667  -24.7167     23.64   
59        59                   bredasdorp -34.5322   20.0403     23.77   
62        62                    al bayda'  32.7627   21.7551     24.03   
77        77            wailua homesteads  22.0669 -159.3780     24.44   
86        86                       hirara  24.8000  125.2833     24.15   
90        90                    carnarvon -24.8667  113.6333     22.02   
93        93               port elizabeth -33.9180   25.5701     23.94   
110      110                   nouadhibou  20.9310  -17.0347     21.99   
115      115                       kourou   5.1552  -52.6478     26.03   
124      124                       has

### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Create a copy of the filtered_cities DataFrame and select the columns you need
hotel_df = filtered_cities[[
    "City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,alvaraes,BR,-3.2208,-64.8042,95,
36,tazacorte,ES,28.6290,-17.9293,82,
46,tezu,IN,27.9167,96.1667,73,
58,nova sintra,CV,14.8667,-24.7167,72,
59,bredasdorp,ZA,-34.5322,20.0403,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
import requests

# Set your Geoapify API key
api_key = geoapify_key

# Set the base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)
categories = "accommodation.hotel"
limit = 1  # Limit to one result (the nearest hotel)

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Construct the API request URL with the specific city's coordinates
    url = base_url + "?categories=" + categories + "&filter=circle:" + str(lng) + "," + str(lat) + "," + str(
        radius) + "&bias=proximity:" + str(lng) + "," + str(lat) + "&limit=" + str(limit) + "&apiKey=" + api_key

    # Make an API request using the constructed URL
    response = requests.get(url)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.at[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.at[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.at[index, 'City']
             } - nearest hotel: {hotel_df.at[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
alvaraes - nearest hotel: Hotel Divino
tazacorte - nearest hotel: App Leyma
tezu - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
bredasdorp - nearest hotel: Victoria Hotel
al bayda' - nearest hotel: فندق الهديل
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
hirara - nearest hotel: 古謝そば屋
carnarvon - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
nouadhibou - nearest hotel: Hotel valencia
kourou - nearest hotel: SCI Horizon
hasaki - nearest hotel: 大新旅館
soe - nearest hotel: DENA
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
arhribs - nearest hotel: No hotel found
naze - nearest hotel: ビジネス旅館畠山
tamanrasset - nearest hotel: فندق الأمان
lihue - nearest hotel: Kauai Palms
lichinga - nearest hotel: Residencial 2+1
annau - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
alghero - nearest hotel: Catalunya
arraial do cabo - nearest hotel: No hotel found
cova f

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE


import geoviews.tile_sources as gvts


pd.options.plotting.backend = "holoviews"

# Assuming you have a DataFrame 'city_data_df' with columns "Lng", "Lat", "Humidity", "City", and "Country"
df = pd.DataFrame(
    hotel_df,
    columns=[
        "Lng",
        "Lat",
        "Humidity",
        "City",
        "Country",
        "Hotel Name"  # Assuming you have a "Country" column
    ],
)

tile_options = {"width": 700, "height": 500, "xaxis": None, "yaxis": None}
world = gvts.OSM.opts(**tile_options)

# Calculate alpha values based on humidity
df["alpha"] = df["Humidity"] / 100  # Adjust scaling as needed

world * df.hvplot.points(
    geo=True, s="Humidity", c="City", alpha="alpha", hover_cols=["City", "Humidity","Hotel Name","Country"]
)


# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,alpha,Hotel Name,Country)